In [1]:
import requests
import json
import pandas
import copy

In [3]:
url_world_gdpGrowth = "https://www.imf.org/external/datamapper/api/?meta&geoitems&indicators&datasets&values=NGDP_RPCH"
response = requests.get(url_world_gdpGrowth)
jsonString_world_gdpGrowth = response.text
json_world_gdpGrowth = json.loads(jsonString_world_gdpGrowth)
geoitems = json_world_gdpGrowth["geoitems"]

def safe_dict(dictObj, attrName):
    try:
        return dictObj[attrName]
    except:
        return None

name_abbr = [{
        "abbr": geoitem,
        "name": safe_dict(geoitems[geoitem], "label"),
        "type": safe_dict(geoitems[geoitem], "type")
    } for geoitem in geoitems
]

pandas.DataFrame(name_abbr).to_csv("Data/name_abbr.csv", index = False, encoding = "ISO-8859-1")

In [3]:
url_world_map = "https://www.imf.org/external/datamapper/config/world.json"
response = requests.get(url_world_map)
jsonString_world_map = response.text
json_world_map = json.loads(jsonString_world_map)
areas, masses = json_world_map["items"], json_world_map["masses"]

max_x, max_y = 0, 0
for mass in masses:
    for point in masses[mass]["poly"]:
        max_x = max(max_x, point[0])
        max_y = max(max_y, point[1])

bbox = {
    "Top": 83.567272,
    "Left": -168.965424,
    "Buttom": -55.899163,
    "Right": -169.755329
}

def projectSeries(value, oldStart, oldEnd, newStart, newEnd):
    oldLen = oldEnd - oldStart
    newLen = newEnd - newStart
    return newLen / oldLen * (value - oldEnd) + newEnd

def projectionAtX(point):
    #print(point[0], end = " -> ")
    xLen1 = 0 - bbox["Left"]
    xLen2 = 180
    xLen3 = bbox["Right"] + 180
    xLen = xLen1 + xLen2 + xLen3
    if point[0] < xLen1 / xLen * 200:
        point[0] = projectSeries(point[0], 0, xLen1 / xLen * 200, bbox["Left"], 0)
    elif point[0] < (xLen2 + xLen1) / xLen * 200:
        point[0] = projectSeries(point[0], xLen1 / xLen * 200, (xLen2 + xLen1) / xLen * 200, 0, 180)
    else:
        #point[0] = projectSeries(point[0], (xLen2 + xLen1) / xLen * 200, 200, 180, bbox["Right"] + 360)
        point[0] = projectSeries(point[0], (xLen2 + xLen1) / xLen * 200, 200, -180, bbox["Right"])
    return point

def projectionAtY(point):
    point[1] = projectSeries(point[1], 0, max_y, bbox["Top"], bbox["Buttom"])
    return point

def projection(points, name):
    #print(name)
    return [projectionAtX(projectionAtY(point)) for point in points]
        
def safe_polygon(polygon_coordinates, id = None, name = None):
    if polygon_coordinates[0] != polygon_coordinates[-1]:
        print("id: {}, name: {}, first: {}, last: {}".format(
            id, name, polygon_coordinates[0], polygon_coordinates[-1]
        ))
        polygon_coordinates.append(copy.deepcopy(polygon_coordinates[0]))
    return polygon_coordinates

def split_polygon(polygons):
    for polygon in polygons:
        need_split = False
        for coord_id in range(1, len(polygon)):
            need_split = need_split or polygon[0][0] * polygon[coord_id][0] < 0
        if need_split:
            stack = []
            coord_id = 1
            while coord_id < len(polygon):
                if polygon[0][0] * polygon[coord_id][0] < 0:
                    while coord_id < len(polygon) and polygon[0][0] * polygon[coord_id][0] < 0:
                        stack.append(polygon.pop(coord_id))
                        coord_id += 1
                    polygons.append(copy.deepcopy(stack))
                    stack = []
                    print("Hi!")
                else:
                    coord_id += 1
    return polygons

geoJson_world_map = {
    "type": "FeatureCollection",
    "features": [{
        "type": "Feature",
        "properties": {"name": area_name},
        "geometry": {
            "type": "MultiPolygon",
            "coordinates": [
                [
                    projection(
                        safe_polygon(
                            masses[str(area_polygons_id)]["poly"], area_polygons_id, area_name), area_name
                    )
                ]
                for area_polygons_id 
                in areas[area_name]["m"]
            ]
        }
    } for area_name in areas]
}

with open("Data/world_map.json", "w") as jsonFile:
    json.dump(geoJson_world_map, jsonFile, indent = None)

id: 90, name: SSD, first: [113.058, 62.551], last: [113.06, 62.551]
id: 91, name: SDN, first: [114.613, 53.89], last: [114.604, 53.885]
id: 490, name: EGY, first: [114.604, 53.885], last: [114.613, 53.89]
id: 671, name: SUR, first: [62.614, 67.522], last: [62.614, 67.528]
